<a href="https://colab.research.google.com/github/DmitryKutsev/NIS_SentiFrame/blob/master/parsing_bl_pandas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install pymorphy2[fast]

In [0]:
import pandas as pd
import json
from collections import Counter

# для переноса джейсона в датафрейм
from pandas.io.json import json_normalize

from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()

In [0]:
! wget https://raw.githubusercontent.com/nicolay-r/RuSentiFrames/master/collection.json

In [0]:
# загружаем базовый лексикон
with open ('collection.json') as jf:
  bl_data = json.load(jf)

In [5]:
# задаём названия колонок, хотя этого можно и не делать, наверное
cols = ['title', 'variants', 'comment', 'roles.a0', 'roles.a1', 'roles.a2', 'roles.a3',
       'frames.polarity', 'frames.effect', 'frames.state', 'variant', 'key']

# создаём шаблон итогового датафрейма с нужными колонками (см. выше)
# к нему будем приклеивать датафреймы, в которых строки будут различаться лишь
# колонкой вариант
bl_df = pd.DataFrame(columns=cols)

# для каждого фрейма
for fr in bl_data:
  # достаём фрейм
  frame = bl_data[fr]
  # создаём второй шаблон, к которому будем приклеивать строки для каждого варианта
  # когда наполним этот второй шаблон, он отправится в итоговый дф
  df = pd.DataFrame(columns=cols)


  # для каждого варианта предиката
  for variant in frame['variants']:
    # переносим джейсон фрейма, с которым работаем, в датафрейм из одной строки
    # max_level=1 потому что дальше фрейм не разворачивается
    base_df = json_normalize(frame, max_level=1)
    # в новую колонку кладём вариант
    base_df['variant'] = [variant]
    # ко второму дф приклеиваем дф-строку с вариантом, индекс сбрасываем
    # не сортируем, чтоб нимношк быстрее было
    df = df.append(base_df, ignore_index=True, sort=False)


  # на этом этапе у нас df для фрейма наполнился одинаковыми строками
  # где только варианты различаются
  # ключ, соответствующий фрейму, лежит в fr
  # до кучи положим его туда же, в каждую строку
  df['key'] = [fr for i in range(0,len(frame['variants']))]
  # получившийся дф для фрейма приклеиваем к итоговому
  bl_df = bl_df.append(df, ignore_index=True, sort=False)
  # идём в следующий фрейм

# смотрим, сколько строчек вышло в дф и не потеряли ли мы что-то
len(bl_df)

6247

In [0]:
# дропнем лишние колонки
# хотя можно и не дропать
# две колонки комментариев хорошо бы было совместить конечно и оставить
bl_df.drop(['comment', 'comments', 'frames.value', 'title', 'variants'], axis=1, inplace=True)

# вот такие колонки остались на этом этапе
# bl_df.columns
# ['roles.a0', 'roles.a1', 'roles.a2', 'roles.a3', 'frames.polarity', 'frames.effect', 'frames.state', 'variant', 'key']
# многие ячейки заполнены NaN-ами

In [0]:
# # это всё неважно, поэтому закаменчено

# # посмотрим, какие эттитьюдс бывают вообще в базовом лексиконе
# pol_counter = Counter()
# # для листа полярностей из каждой строчки дф (этот лист хранится в колонке frames.polarity)
# for pol_lst in bl_df[
#                      # (смотрим только на те строки, где полярити вообще есть)
#                      bl_df['frames.polarity'].notnull()
#                      ]['frames.polarity'].values:
#   # для отдельной полярности в этом листе (которая как тип тоже лист)
#   for pol in pol_lst:
#     # кладём двух склеенных через пробел участников как строку в каунтер
#     pol_counter.update([' '.join([str(el) for el in pol])])
# # смотрим, что вышло
# pol_counter

In [0]:
# # это тоже неважно
# #  но прикольно, что в каком-то фрейме аж 11 полярностей

# # добавим колонку с количеством полярностей
# bl_df['n_polarities'] = bl_df['frames.polarity'].apply(lambda x: len(x) if type(x)==list else 0)
# # посмотрим на то, сколько полярностей может быть во фрейме
# Counter(bl_df['n_polarities'].values)

In [0]:
# # а в этих фреймах вообще нет эффекта
# set(bl_df[bl_df['frames.effect'].isnull()]['key'].values)

In [0]:
# # а это какие бывают эффекты

# effect_counter = Counter()
# for effect_lst in bl_df[bl_df['frames.effect'].notnull()]['frames.effect'].values:
#   for effect in effect_lst:
#     effect_counter.update([' '.join([str(el) for el in effect])])
# effect_counter

In [0]:
# # можно запринтить все возможные пары отношений
# но это не особо интересно

# possible_attitudes_set = set(['_'.join(pol.split()[0:2]) for pol in pol_counter])
# for attitude in possible_attitudes_set:
#   print (attitude)

In [54]:
# посмотрим, сколько всего повторяющихся вариантов предикатов
# их 196

bl_df.variant.value_counts().head(197)

прослеживать        3
обставлять          3
снижение            3
проследить          3
колоть              3
                   ..
углубить            2
замарать            2
расстроиться        2
следовать           2
применить оружие    1
Name: variant, Length: 197, dtype: int64

In [0]:
# ну, терь самое главное (типа)

# идем по строкам
for row in bl_df.itertuples():
  # достаём индес строки
  row_id = row[0]
  # и достаём из неё лист полярностей
  polarities_list = row[5]

  # пытаемся
  try:
    # для отдельной полярности
    for polarity in polarities_list:

      # обращаемся к элементам полярности (которая сама список) по индексу
      # элемент с индексом 0 - это тот, кто относится
      # элемент  индексом 1 - тот, к кому относятся
      # создаём название колонки формата 'КтоОтносится_ККому'
      col = '{}_{}'.format(polarity[0], polarity[1])

      #  обращаемся к ячейке по адресу [индекс строки, название новой колонки]
      # заполняем эту ячейку меткой (метка - это элемент полярности с индексом 2)
      # ну, метка бывает pos или neg
      # степень уверенности не берём с собой
      bl_df.loc[row_id, col] = polarity[2]


      # чтоб сохранить меру уверенности, можно её приклеить к метке,
      # если заменить строку выше на строку ниже
      # но в данный момент неясно, зачем она нужна, так что не будем
      # bl_df.loc[row_id, col] = '{} {}'.format(polarity[2], polarity[3])
  
  # если попытка не удалась, значит в полярности NaN
  except:
    pass
  # идём в следующую строку

#Достаём списком

In [0]:
# тут по условию достаётся список глаголов
# раскаментим одно из двух условий ниже

condition = 'pos'
# condition = 'neg'

# по условиям достаём варианты
variants = bl_df[
      
      # первое условие
      (bl_df.a0_a1 == condition)
      
      # логическое И
      &
      
      # второе условие
      (bl_df.a1_a0 == condition)

      # можно добавить ещё миллион каких-нить условий вот в таком формате
      # &
      # (bl_df.a1_a0 == condition)

]['variant'].values

# нам интересны только те предикаты, которым пайморфи даёт pos-тег инфинитива
# или финитного глагола (на всякий)
# ну и чтобы предикат был одним словом, никаких пробелов
candidates = [variant for variant in variants 
              if 
              (' ' not in variant) 
              and 
              (('INFN' or 'VERB') in morph.parse(variant)[0].tag)
              ]
# имя файла будет с интересующей нас меткой в начале
fname = "{}_candidates_list.json".format(condition)

# сбрасываем лист кандидатов туда
with open(fname, "w") as write_file:
    json.dump(candidates, write_file, 
              # это чтоб нормально кириллица записалась
              ensure_ascii=False
              )

In [0]:
# делаем допущение, что так у нас получится вытащить глаголы с аргументом в дативе
# и второе допущение - что такие аргументы лежат в а1
# на самом деле мы поэкспериментировали с а0 и а2 и a3 тоже
# эксперименты показали, что лучше их в a1 искать

# заводим новую колонку 'to_whom'
# в ней будем хранить тру и фолс
# тру - мб датив есть
# фолс - мб его нету

# смотрим на описание роли a1
# если в нём есть ', кому' ИЛИ ', чему', лямбда-функция возвращает True
# и кладёт его в новую колонку
# ну и кладёт фолс если нету ни одной из этих строк в описании
# такого аргумента может и не быть, поэтому на всякий конвертируем его в строку
# эксепшены я не умею делать! научите...
bl_df['to_whom'] = bl_df['roles.a1'].apply(lambda x: (', кому' or ', чему') in str(x))

# в список кандидатов на датив кладём только нужные
# отсеиваем нужные по условиям
dative_candidates = [variant for variant
                    # во-первых, чтобы тру лежало в 'to_whom'
                     in bl_df[bl_df['to_whom'] == True]['variant'].values 
                    # во-вторых, чтоб это был глагол
                     if (' ' not in variant) and (('INFN' or 'VERB') in morph.parse(variant)[0].tag)]

# сбрасываем получившийся список в файл
with open("dative_candidates_list.json", "w") as write_file:
    json.dump(dative_candidates, write_file, ensure_ascii=False)

#Достаём словарём

In [0]:
# таким же образом достаём словарь вида {'глагол':'список полярностей'}
# если попадаются дубликаты, записывается полярность последнего

condition = 'pos'
# condition = 'neg'

conditioned_df = bl_df[
      (bl_df.a0_a1 == condition)
      &
      (bl_df.a1_a0 == condition)
      # можно добавить ещё миллион каких-нить условий вот в таком формате
      # &
      # (bl_df.a1_a0 == condition)
]

variants = conditioned_df['variant'].values
pols = conditioned_df['frames.polarity'].values

candidates = {k:v for (k,v) in dict(zip(variants, pols)).items() 
              if (' ' not in k)
              and (('INFN' or 'VERB') in morph.parse(k)[0].tag)
              }

fname = "{}_candidates_dict.json".format(condition)

with open(fname, "w") as write_file:
    json.dump(candidates, write_file, ensure_ascii=False)

In [0]:
# dictionary of dative candidates

bl_df['to_whom'] = bl_df['roles.a1'].apply(lambda x: (', кому' or ', чему') in str(x))

conditioned_df = bl_df[bl_df['to_whom'] == True]

variants = conditioned_df['variant'].values
pols = conditioned_df['frames.polarity'].values

candidates = {k:v for (k,v) in dict(zip(variants, pols)).items() 
              if (' ' not in k)
              and (('INFN' or 'VERB') in morph.parse(k)[0].tag)
              }

# сбрасываем получившийся список в файл
with open("dative_candidates_dict.json", "w") as write_file:
    json.dump(candidates, write_file, ensure_ascii=False)

# Пикл

In [0]:
# # в пикл записываем подопытный датафрейм
# # и скачиваем
# # чтоб потом его просто загрузить
# # но это не обязательно
# bl_df.to_pickle('bl_df.pkl')